In [1]:
import numpy as np
import tensorflow as tf

# 1. 基本文本处理技能

1.1 分词的概念（分词的正向最大、逆向最大、双向最大匹配法）；

分词的基本原则：

1、颗粒度越大越好；2、切分结果中非词典词越少越好，单字字典词数越少越好；3、总体词数越少越好，在相同字数的情况下，总词数越少，说明语义单元越少，那么相对的单个语义单元的权重会越大，因此准确性会越高。

正向最大匹配法：是指以词典为依据，正向取词的方法.以下简单编辑实现

In [12]:
list1 = ['我','爱','中国','中国共产党','共产党']
str1 ='我爱中国共产党'

In [13]:
a = []
while len(str1):
    for i in range(len(str1)):
        if str1[:len(str1)-i] in list1:
            a.append(str1[:len(str1)-i])
            str1 = str1[len(str1)-i:]


In [14]:
a

['我', '爱', '中国共产党']

逆向匹配法：是指以词典为依据，反向取词的方法.
双向最大匹配法：两种算法都切一遍，然后根据大颗粒度词越多越好，非词典词和单字词越少越好的原则，选取其中一种分词结果输出。

# 1.2 词、字符频率统计

In [16]:
data = '研究任何过程，如果是存在着两个以上矛盾过程的话，就要用力找出它的主要矛盾，捉住了这个主要矛盾，\
一切问题就迎刃而解了一切反动派都是纸老虎。看起来，反动派的样子是可怕的，但是实际上并没有什么了不起的力量。\
从长远的观点看问题，真正强大的力量不是属于反动派，而是属于人民。'

In [17]:
import jieba

In [18]:
V = jieba.lcut(data)
print(V)

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/dn/fjsmvq4j5rlfstkkm70jp8b40000gn/T/jieba.cache
Loading model cost 1.536 seconds.
Prefix dict has been built succesfully.


['研究', '任何', '过程', '，', '如果', '是', '存在', '着', '两个', '以上', '矛盾', '过程', '的话', '，', '就要', '用力', '找出', '它', '的', '主要矛盾', '，', '捉住', '了', '这个', '主要矛盾', '，', '一切', '问题', '就', '迎刃而解', '了', '一切', '反动派', '都', '是', '纸老虎', '。', '看起来', '，', '反动派', '的', '样子', '是', '可怕', '的', '，', '但是', '实际上', '并', '没有', '什么', '了不起', '的', '力量', '。', '从', '长远', '的', '观点', '看', '问题', '，', '真正', '强大', '的', '力量', '不是', '属于', '反动派', '，', '而是', '属于', '人民', '。']


使用pandas库进行统计

In [23]:
import pandas as pd
import numpy as np

In [24]:
df = pd.DataFrame(V,columns=['A'])

In [25]:
words=df.groupby(by=['A'])['A'].agg({"计数":np.size})

/Users/app/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


In [28]:
words1=words.reset_index().sort_values(by="计数",ascending=False)

In [29]:
words1.head()

,A,计数
48,，,8
31,的,6
14,反动派,3
27,是,3
0,。,3


另一种方法

In [30]:
from collections import Counter

In [31]:
B = Counter(V)
print(B)

Counter({'，': 8, '的': 6, '是': 3, '反动派': 3, '。': 3, '过程': 2, '主要矛盾': 2, '了': 2, '一切': 2, '问题': 2, '力量': 2, '属于': 2, '研究': 1, '任何': 1, '如果': 1, '存在': 1, '着': 1, '两个': 1, '以上': 1, '矛盾': 1, '的话': 1, '就要': 1, '用力': 1, '找出': 1, '它': 1, '捉住': 1, '这个': 1, '就': 1, '迎刃而解': 1, '都': 1, '纸老虎': 1, '看起来': 1, '样子': 1, '可怕': 1, '但是': 1, '实际上': 1, '并': 1, '没有': 1, '什么': 1, '了不起': 1, '从': 1, '长远': 1, '观点': 1, '看': 1, '真正': 1, '强大': 1, '不是': 1, '而是': 1, '人民': 1})


# 2. 概念

# 2.1 语言模型中unigram、bigram、trigram的概念

N-Gram（有时也称为N元模型）是自然语言处理中一个非常重要的概念，通常在NLP中，
人们基于一定的语料库，可以利用N-Gram来预计或者评估一个句子是否合理。

个人理解的就是马尔可夫链，1元为iid，1阶马尔可夫对应2元，2阶对应3元，依照这个规则计算条件概率啦。

具体使用到橘子，判断橘子是否合理就是所谓的因子分解，然后从语料库中的历史数据，近似计算概率

# 3. 文本矩阵化：要求采用词袋模型且是词级别的矩阵化

In [33]:
data_1 = "为了让大家学过之后有实效，这个微信群制定了几项群规：每天早上有群员诵读书中的章节，每晚固定时间有人来做导读分享，\
群成员自己学习后还要写读书札记。群里实行严格的考勤制度，如果参与分享、撰写札记的次数达不到标准，则会被移除。\
群里有专门的管理员负责安排大家的学习进度、整理学习成果。赵晋泰告诉本报记者：“订群规就是立规矩，目的是一开始就告诉大家，\
我们要确保学习有效果，而不是做个噱头，流于形式。天津外国语大学哲学研究所副教授吴倩就在学习交流群中。遵循群规学习，让她收获不少。\
“开始时，清晨诵读只有少数几位同仁参与，后来慢慢越来越多的朋友加入进来。起初还不觉得什么，养成习惯以后，一边吃早餐一边听大家语调铿锵地读书，\
让人不由地振奋，整个早晨都是元气满满的。从学习效果来看，静静看书与大声诵读的感受截然不同，前者是静默而思，后者则是在振奋中前行，一字一句品读\
书中原文的文字魅力，鼓舞了自己也感染了别人。晚间的导读一般由一位做相关学术研究或实际工作涉及相关领域的群友展开，有的导读是理论探析，有的是\
经验阐释。听完导读，大家会根据总书记的这篇讲话进行探讨切磋，不仅深入理解了理论，而且结下了深厚的共学情谊。"

In [34]:
import re

In [35]:
data_all = re.findall(r'[\u4e00-\u9fa5]+', data_1)

In [38]:
data_all

['为了让大家学过之后有实效',
 '这个微信群制定了几项群规',
 '每天早上有群员诵读书中的章节',
 '每晚固定时间有人来做导读分享',
 '群成员自己学习后还要写读书札记',
 '群里实行严格的考勤制度',
 '如果参与分享',
 '撰写札记的次数达不到标准',
 '则会被移除',
 '群里有专门的管理员负责安排大家的学习进度',
 '整理学习成果',
 '赵晋泰告诉本报记者',
 '订群规就是立规矩',
 '目的是一开始就告诉大家',
 '我们要确保学习有效果',
 '而不是做个噱头',
 '流于形式',
 '天津外国语大学哲学研究所副教授吴倩就在学习交流群中',
 '遵循群规学习',
 '让她收获不少',
 '开始时',
 '清晨诵读只有少数几位同仁参与',
 '后来慢慢越来越多的朋友加入进来',
 '起初还不觉得什么',
 '养成习惯以后',
 '一边吃早餐一边听大家语调铿锵地读书',
 '让人不由地振奋',
 '整个早晨都是元气满满的',
 '从学习效果来看',
 '静静看书与大声诵读的感受截然不同',
 '前者是静默而思',
 '后者则是在振奋中前行',
 '一字一句品读书中原文的文字魅力',
 '鼓舞了自己也感染了别人',
 '晚间的导读一般由一位做相关学术研究或实际工作涉及相关领域的群友展开',
 '有的导读是理论探析',
 '有的是经验阐释',
 '听完导读',
 '大家会根据总书记的这篇讲话进行探讨切磋',
 '不仅深入理解了理论',
 '而且结下了深厚的共学情谊']

In [40]:
word_all = [jieba.lcut(i) for i in data_all]

In [41]:
word_all

[['为了', '让', '大家', '学过', '之后', '有', '实效'],
 ['这个', '微信群', '制定', '了', '几项', '群规'],
 ['每天', '早上', '有', '群员', '诵读', '书中', '的', '章节'],
 ['每晚', '固定', '时间', '有人', '来', '做', '导读', '分享'],
 ['群', '成员', '自己', '学习', '后', '还要', '写', '读书', '札记'],
 ['群里', '实行', '严格', '的', '考勤制度'],
 ['如果', '参与', '分享'],
 ['撰写', '札记', '的', '次数', '达', '不到', '标准'],
 ['则', '会', '被', '移除'],
 ['群里', '有', '专门', '的', '管理员', '负责', '安排', '大家', '的', '学习', '进度'],
 ['整理', '学习', '成果'],
 ['赵晋', '泰', '告诉', '本报记者'],
 ['订群规', '就是', '立', '规矩'],
 ['目的', '是', '一', '开始', '就', '告诉', '大家'],
 ['我们', '要', '确保', '学习', '有', '效果'],
 ['而', '不是', '做个', '噱头'],
 ['流于形式'],
 ['天津', '外国语', '大学', '哲学', '研究所', '副教授', '吴倩', '就', '在', '学习', '交流', '群中'],
 ['遵循', '群规', '学习'],
 ['让', '她', '收获', '不少'],
 ['开始', '时'],
 ['清晨', '诵读', '只有', '少数几位', '同仁', '参与'],
 ['后来', '慢慢', '越来越', '多', '的', '朋友', '加入', '进来'],
 ['起初', '还', '不', '觉得', '什么'],
 ['养成', '习惯', '以后'],
 ['一边', '吃', '早餐', '一边', '听', '大家', '语调', '铿锵', '地', '读书'],
 ['让', '人', '不由', '地', '振奋'],
 ['整个', '早晨', '都

In [42]:
from sklearn.feature_extraction.text import CountVectorizer

In [43]:
count = CountVectorizer()

In [59]:
for i in range(len(word_all)):
    if i == 0 :
        a = word_all[i]
    else:
        a = a+word_all[i]

In [61]:
a_set = set(a)

In [63]:
X = count.fit(a_set)

In [68]:
for i in word_all:
    print(count.transform(i).toarray())###显示矩阵

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
  0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 